<a href="https://colab.research.google.com/github/pacastl/PruebaIA/blob/main/Selecci%C3%B3n_de_descriptores_y_an%C3%A1lisis_estad%C3%ADstico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Selección de descriptores

In [1]:
import numpy as np
import pandas as pd
# url del repositorio de github para poder leerlos
url_train = 'https://raw.githubusercontent.com/pacastl/PruebaIA/main/train.csv'
url_test = 'https://raw.githubusercontent.com/pacastl/PruebaIA/main/test.csv'

# Leemos los CSV como Pandas DataFrames
df = pd.read_csv(url_train)
#test_df = pd.read_csv(url_test)

#otra forma de leerlos porque los descargué en mi pc
train_file = r'C:\Users\Pablo\Downloads\train.csv'
test_file = r'C:\Users\Pablo\Downloads\test.csv'

#df = pd.read_csv(train_file)
#test_df = pd.read_csv(test_file)
# Mostramos los datos para comprobar que se han leído
print(df.head())
#print(test_df.head())

         SL      EEG  BP   HR  CIRCULATION  ACTIVITY
0   4019.64 -1600.00  13   79          317         3
1   2191.03 -1146.08  20   54          165         2
2   2787.99 -1263.38  46   67          224         2
3   9545.98 -2848.93  26  138          554         4
4  14148.80 -2381.15  85  120          809         4


Importamos la librería y separamos las variables predictoras y la variable objetivo:

In [2]:
from sklearn.feature_selection import SelectKBest, f_regression

X = df.drop('ACTIVITY', axis=1)
y = df['ACTIVITY']


usamos SelectKBest con f_regression como función de puntuación para seleccionar las 3 mejores

In [3]:
selector = SelectKBest(score_func=f_regression, k=3)
X_new = selector.fit_transform(X, y)


In [4]:
scores = selector.scores_
selected_features = X.columns[selector.get_support()]

print('Selected Features:', selected_features)
print('Scores:', scores[selector.get_support()])


Selected Features: Index(['SL', 'HR', 'CIRCULATION'], dtype='object')
Scores: [279.49121672 265.11169006 230.5013809 ]


Los puntajes indican la importancia de cada característica en la tarea de clasificación. En este caso, la característica con el puntaje más alto es 'SL', seguida de 'HR' y luego 'CIRCULATION'.

Se pueden utilizar las tres características para construir un modelo de clasificación, y se puede esperar que la característica 'SL' tenga el mayor impacto en la predicción del modelo

# Análisis estadístico

### 5. Estudio estadístico

Lo último que vamos a ver en esta parte es cómo realizar un estudio estadístico basado en lo que vimos en clase, para ello utilizaremos la función `compare_methods` disponible en el módulo StatisticalAnalysis. Comienza instalando este módulo usando `pip`.

In [ ]:
!pip install StatisticalAnalysis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from StatisticalAnalysis import compare_methods

Para utilizar este método debemos comenzar indicando los algoritmos a utilizar y las distribuciones de los hiperparámetros a optimizar. Vamos a utilizar los 5 modelos vistos hasta ahora: árboles de decisión, SVMs, KNN, Regresión logística y red neuronal, así que definimos estos modelos y los parámetros a optimizar.

In [ ]:
# Árbol de decisión
clfTree = DecisionTreeClassifier(random_state=84)
param_distTree = {"min_samples_split": sp_randint(3, 30)}
# SVM
clfSVC = SVC(random_state=84)
param_distSVC = {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],'kernel': ['rbf'], 'class_weight':['balanced', None]}
# KNN
clfKNN = KNeighborsClassifier()
param_distKNN = {'n_neighbors':sp_randint(3, 30)}
# Regresión logística
clfLR = LogisticRegression(random_state=84)
param_distLR = {'C': [0.1,0.5,1, 10, 100, 1000]}
# Red neuronal
clfMLP = MLPClassifier(random_state=84)
param_distMLP = {'activation': ['identity', 'logistic', 'tanh', 'relu'], 'solver': ['lbfgs', 'sgd', 'adam'],
                 'alpha': sp_randint(0.0001, 1), 'learning_rate': ['constant', 'invscaling', 'adaptive'],
                 'hidden_layer_sizes': [(5,2), (3,3,3), (5,3,2), (5,4,3,2)],
                 'momentum': [0.9, 0.95, 0.99]}

Creamos tres listas que contendrán respectivamente:

(1) Los algoritmos:

In [ ]:
listAlgorithms = [clfTree,clfSVC,clfKNN,clfLR,clfMLP]

(2) Los parámetros a optimizar:

In [ ]:
listParams = [param_distTree,param_distSVC,param_distKNN,param_distLR,param_distMLP]

(3) Los nombres de los algoritmos:

In [ ]:
listNames = ["Arbol", "SVM", "KNN", "LR", "MLP"]

Invocamos a la función `compare_methods` pasándole el dataset completo, el conjunto de etiquetas completo, las tres listas definidas anteriormente, y la métrica. Los valores válidos
para la métrica son accuracy, precision, recall, f1 o auroc. Al invocar a la función anterior se producirá un informe de resultados. 

In [7]:
pip install StatisticalAnalysis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for StatisticalAnalysis: filename=StatisticalAnalysis-0.0.5-py2.py3-none-any.whl size=13119 sha256=e38c517e860903a6f2d3cb15bcafc0af3d6870bafc5f6ea83318e77c77e8e361
  Stored in directory: /root/.cache/pip/wheels/f4/ef/8b/a5bb9190fa93ff8e8031232c0cf16c6da6a0d30a7d491ceb46
Successfully built StatisticalAnalysis


In [11]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from scipy.stats import randint as sp_randint
from StatisticalAnalysis import compare_methods


# define features (X) and target variable (Y)
X = df.drop('ACTIVITY', axis=1).values
Y = df['ACTIVITY'].values

# Árbol de decisión
clfTree = DecisionTreeClassifier(random_state=84)
param_distTree = {"min_samples_split": sp_randint(3, 30)}

# SVM
clfSVC = SVC(random_state=84)
param_distSVC = {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],'kernel': ['rbf'], 'class_weight':['balanced', None]}

# KNN
clfKNN = KNeighborsClassifier()
param_distKNN = {'n_neighbors':sp_randint(3, 30)}

# Regresión logística
clfLR = LogisticRegression(random_state=84)
param_distLR = {'C': [0.1,0.5,1, 10, 100, 1000]}

# Red neuronal
clfMLP = MLPClassifier(random_state=84)
param_distMLP = {'activation': ['identity', 'logistic', 'tanh', 'relu'], 'solver': ['lbfgs', 'sgd', 'adam'],
                 'alpha': sp_randint(1, 1000), 'learning_rate': ['constant', 'invscaling', 'adaptive'],
                 'hidden_layer_sizes': [(5,2), (3,3,3), (5,3,2), (5,4,3,2)],
                 'momentum': [0.9, 0.95, 0.99]}

# Creamos tres listas que contendrán respectivamente:
# (1) Los algoritmos:
listAlgorithms = [clfTree,clfSVC,clfKNN,clfLR,clfMLP]
# (2) Los parámetros a optimizar:
listParams = [param_distTree,param_distSVC,param_distKNN,param_distLR,param_distMLP]
# (3) Los nombres de los algoritmos:
listNames = ["Arbol", "KNN", "MLP"]

compare_methods(X, Y, listAlgorithms, listParams, listNames, metric='auroc')


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 16 is smaller than n_iter=20. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
Process ForkPoolWorker-16:
Process ForkPoolWorker-17:
Process ForkPoolWorker-19:
Process ForkPoolWorker-20:
Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.9/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()

KeyboardInterrupt: ignored

  File "/usr/local/lib/python3.9/dist-packages/joblib/parallel.py", line 1088, in __call__
    while self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.9/dist-packages/joblib/parallel.py", line 819, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/usr/local/lib/python3.9/dist-packages/joblib/parallel.py", line 1088, in __call__
    while self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.9/dist-packages/joblib/parallel.py", line 1088, in __call__
    while self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.9/dist-packages/joblib/parallel.py", line 901, in dispatch_one_batch
    self._dispatch(tasks)
  File "/usr/local/lib/python3.9/dist-packages/joblib/_parallel_backends.py", line 208, in apply_async
    result = ImmediateResult(func)
  File "/usr/local/lib/python3.9/dist-packages/joblib/parallel.py", line 901, in dispatch_one_batch
    self._dispatch(tasks)
  File "/usr/local/lib/python3.9/dist-packages/joblib/p

##### Preguntas

¿Qué tipo de tests se ha aplicado? ¿Paramétrico o no paramétrico? ¿por qué?

Respuesta: Como no se cumplía las condiciones necesarias para el parámetrico--> se aplica NO paramétrico

¿Cuál ha sido el modelo ganador?

Respuesta: Regresión Logística

¿Hay diferencias significativas con los otros modelos?

Respuesta: Sí, los resultados son bastante mejores.